# Talks analyzer

## Introduction
This project focuses on analyzing the speeches given during <b>TEDx</b> events. I will analyze both the emotions expressed during the speech and correlate them with the sentences spoken. Additionally, comments will be extracted to understand the listeners' perspectives. Additionally, a summary of the TEDx talk will be provided, along with five possible questions related to the speech that could be asked to the speaker.

### What is a TEDx?
<b>TED (Technology Entertainment Design)</b> is a series of conferences, also known as TED talks, managed by the American non-profit organization Sapling Foundation.

TED, which began in February 1984 as a single event and became an annual conference in 1990, was initially focused on technology and design, consistent with its origins in Silicon Valley, but later expanded its scope to include the scientific, cultural, and academic worlds.

## Where did I extract the data from?
The data we will use was extracted through Python scripts and Google's APIs, specifically YouTube's.
First, I performed a search on the topic of interest, in this case, <b>TEDx</b>, using an API call, and extracted the IDs of several videos.
Using a function, I created the complete video URL to leverage a Python library, <b>PyTubeFix</b>, which allowed me to download the videos using the URL. 
Once the videos were downloaded, I extracted the audio in <b>wav</b> format using the MoviePy library.

In [ ]:
import argparse
from pytubefix import YouTube
from pytubefix.cli import on_progress
from googleExample import youtube_search
import os
from extract_audio import extract_audio
from readVideosTitle import leggi_titoli_video

# Where to save data
SAVE_PATH = os.path.expanduser("~/Desktop/Facial-Emotion-Recognition\ base")

def create_link(videoIds):
	links = []
	for videoId in videoIds:
		print(f"Processing video ID: {videoId}")
		link = f"https://www.youtube.com/watch?v={videoId}"
		links.append(link)
	return links

def download_youtube_video(url, save_path='.'):
	try:
		print(f"Attempting to download video from URL: {url}")
		yt = YouTube(url, on_progress_callback = on_progress)

		ys = yt.streams.get_highest_resolution()
		ys.download(output_path='./videos')

	except Exception as e:
		print(f"Errore durante il download: {e}")


In [1]:
# Create an argument parser
parser = argparse.ArgumentParser()
parser.add_argument('--q', help='Search term', default='Google')
parser.add_argument('--max-results', help='Max results', default=5, type=int)

# Instead of parsing command line arguments, set them in a dictionary
args_dict = {'q': 'TEDx', 'max_results': 10}  # Replace with your desired values
args = parser.parse_args(args=[])  # Initialize args
args.__dict__.update(args_dict)  # Update args with your values
os.makedirs('./videos', exist_ok=True)
os.makedirs('./audio', exist_ok=True)
os.makedirs('./transcriptions', exist_ok=True)

videoIds = youtube_search(args)

# Links of videos to download
links = create_link(videoIds)

# Download videos from the list
for link in links:
    download_youtube_video(link, SAVE_PATH)
video_titles = leggi_titoli_video()

for video_title in video_titles:
    extract_audio('./videos/' + video_title.replace(' ', '_') + '.mp4', './audio/' + video_title + '.wav')

Video IDs: ['kKvK2foOTJM', 'WeJrU-VJGfg', '7CBfCW67xT8', 'lXZ5Bo5lafA', 'MLKj1puoWCg', 'PBaFURjVrm0', '3k_wxTkYN50']
Processing video ID: kKvK2foOTJM
Processing video ID: WeJrU-VJGfg
Processing video ID: 7CBfCW67xT8
Processing video ID: lXZ5Bo5lafA
Processing video ID: MLKj1puoWCg
Processing video ID: PBaFURjVrm0
Processing video ID: 3k_wxTkYN50
Links: ['https://www.youtube.com/watch?v=kKvK2foOTJM', 'https://www.youtube.com/watch?v=WeJrU-VJGfg', 'https://www.youtube.com/watch?v=7CBfCW67xT8', 'https://www.youtube.com/watch?v=lXZ5Bo5lafA', 'https://www.youtube.com/watch?v=MLKj1puoWCg', 'https://www.youtube.com/watch?v=PBaFURjVrm0', 'https://www.youtube.com/watch?v=3k_wxTkYN50']
Attempting to download video from URL: https://www.youtube.com/watch?v=kKvK2foOTJM
Attempting to download video from URL: https://www.youtube.com/watch?v=WeJrU-VJGfg
Attempting to download video from URL: https://www.youtube.com/watch?v=7CBfCW67xT8
Attempting to download video from URL: https://www.youtube.com/wat

MoviePy - Done.
MoviePy - Writing audio in ./audio/Brain_Hack_6_secrets_to_learning_faster_backed_by_neuroscience__Lila_Landowski__TEDxHobart.wav


MoviePy - Done.
MoviePy - Writing audio in ./audio/How_to_hack_your_brain_for_better_focus__Sasha_Hamdani__TEDxKC.wav


MoviePy - Done.
MoviePy - Writing audio in ./audio/TEDxOrangeCoast_-_Daniel_Amen_-_Change_Your_Brain_Change_Your_Life.wav


MoviePy - Done.
MoviePy - Writing audio in ./audio/Why_students_should_get_mental_health_days_-_Hailey_Hardcastle_shorts_tedx.wav


MoviePy - Done.
MoviePy - Writing audio in ./audio/How_to_recognize_perfectly_hidden_depression__Margaret_Rutherford__TEDxBocaRaton.wav


MoviePy - Done.
MoviePy - Writing audio in ./audio/The_secret_to_getting_better_sleep_tonight__James_Leinhardt__TEDxManchester.wav


MoviePy - Done.


### Next steps of data extraction
After completing everything, I also extracted some comments from the various videos and saved them in a JSON file.

In [22]:
import os
from googleapiclient.discovery import build
from dotenv import load_dotenv
from googleapiclient.errors import HttpError
from googleExample import youtube_search
# Load environment variables
load_dotenv('.env')
DEVELOPER_KEY = os.getenv('API_KEY')
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

def extract_comments(video_id):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                    developerKey=DEVELOPER_KEY)
    
    #Get the information about the video, including the title
    search_response = youtube.videos().list(
        part='snippet,statistics',
        id=video_id,
    ).execute()
    
    #Initialize variable for video title
    video_title = ""
    
    # Extract the title of the video
    for item in search_response['items']:
        if 'snippet' not in item:
            return None  #If there is no snippet, return None
        video_title = item['snippet']['title']
    
    #Get video comments
    results = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        textFormat="plainText"
    ).execute()

    video_comments = []

    for item in results["items"]:
        comment = item["snippet"]["topLevelComment"]
        author = comment["snippet"]["authorDisplayName"]
        text = comment["snippet"]["textDisplay"]

        video_comments.append({
            "author": author,
            "text": text
        })

    return {
        "videoId": video_id,
        "videoTitle": video_title,
        "comments": video_comments
    }


comments_json = []

for video_id in videoIds:
    video_comments_data = extract_comments(video_id)
    comments_json.append(video_comments_data)

print(comments_json)


[{'videoId': 'kKvK2foOTJM', 'videoTitle': 'Brain Hack: 6 secrets to learning faster, backed by neuroscience | Lila Landowski | TEDxHobart', 'comments': [{'author': '@swatiraj7891', 'text': 'What if am doing online studies 🤔  that screen time can affect my consistency???'}, {'author': '@ffaqihf3608', 'text': 'halo arsy'}, {'author': '@Bogdan-Z1', 'text': "So basically every year or once in a while, a neuroscientist comes and explains the exact same thing over and over again as if it's something new. I remember in 2016 watching a video saying exactly the same things, sleep is important, focus, practice, repetition, breaks, exercise... I watched it because I was about to move to Germany and wanted to learn the language as quick as possible. Also that video I watched was probably 3 years old or even older at the time, can't remember 100%."}, {'author': '@adrianaugaldepartida1813', 'text': '❤'}, {'author': '@sunniesunniee6014', 'text': 'wo'}, {'author': '@Anibazsa', 'text': 'its contemplati

## Emotion Recognition
Now that everything is ready, I can proceed with extracting the emotions and transcribing what is being said. To speed up the process, I created two separate processes.

## What model was used for emotion recognition?

The model used is **emotion-fer-plus**, which is a deep convolutional neural network designed for emotion recognition in faces. It is trained on the **FER+ dataset**, which provides updated labels for the standard Emotion FER dataset. Each image in FER+ has been labeled by 10 crowd-sourced taggers, offering a higher quality ground truth for still image emotion recognition compared to the original FER labels. This approach allows for the estimation of an emotion probability distribution per face, enabling the development of algorithms that can produce statistical distributions or multi-label outputs instead of the traditional single-label output.

## What model was used for the transcription?
The model used is **whisper**, which is an automatic speech recognition (ASR) system trained on 680,000 hours of multilingual and multitask supervised data collected from the web.

In [3]:
from expression_ssd_detect import FER_live_cam
from readVideosTitle import leggi_titoli_video
from transcribe import transcribe_audio
import multiprocessing

if __name__ == "__main__":
	titoli = leggi_titoli_video()

	for titolo in titoli:
		p1 = multiprocessing.Process(target=FER_live_cam, args=(titolo,))
		p2 = multiprocessing.Process(target=transcribe_audio, args=(titolo,))

		p1.start()
		p2.start()

		p1.join()
		p2.join()



objc[52180]: Class SDLApplication is implemented in both /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pygame/.dylibs/libSDL2-2.0.0.dylib (0x12be612c8) and /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ffpyplayer/.dylibs/libSDL2-2.0.0.dylib (0x310685330). One of the two will be used. Which one is undefined.
objc[52180]: Class SDLAppDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pygame/.dylibs/libSDL2-2.0.0.dylib (0x12be61318) and /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ffpyplayer/.dylibs/libSDL2-2.0.0.dylib (0x310685380). One of the two will be used. Which one is undefined.
objc[52180]: Class SDLTranslatorResponder is implemented in both /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pygame/.dylibs/libSDL2-2.0.0.dylib (0x12be61390) and /Library/Frameworks/Python.framework/Versions/3.

2024-08-25 21:52:10.083 Python[52503:7724978] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Detected language in segment 0: en
Detected language in segment 1: en
priors nums:4420
FPS: 8.2
FPS: 31.2
FPS: 64.4
FPS: 84.9
FPS: 46.4
FPS: 41.6
FPS: 70.6
FPS: 51.1
FPS: 77.3
FPS: 80.0
FPS: 76.7
FPS: 52.5
FPS: 88.3
FPS: 81.9
FPS: 63.3
FPS: 83.8
FPS: 62.0
FPS: 90.8
FPS: 80.6
FPS: 73.4
FPS: 85.9
FPS: 92.6
FPS: 75.7
FPS: 87.2
FPS: 79.1
FPS: 105.5
FPS: 86.4
FPS: 94.8
FPS: 95.7
FPS: 83.2
FPS: 56.5
FPS: 91.7
FPS: 81.3
FPS: 92.3
FPS: 87.7
FPS: 89.0
FPS: 86.7
FPS: 70.9
FPS: 88.3
FPS: 88.3
FPS: 86.4
FPS: 96.9
FPS: 93.9
FPS: 90.4
FPS: 87.3
FPS: 93.0
FPS: 94.7
FPS: 94.9
FPS: 90.7
FPS: 86.9
FPS: 91.6
FPS: 91.4
FPS: 89.6
FPS: 36.1
FPS: 95.4
FPS: 92.2
FPS: 87.1
FPS: 88.3
FPS: 56.5
FPS: 84.0
FPS: 89.6
FPS: 88.6
FPS: 89.5
FPS: 87.7
FPS: 88.6
FPS: 90.0
FPS: 91.0
FPS: 81.4
FPS: 81.2
FPS: 82.7
FPS: 91.3
FPS: 89.2
FPS: 92.1
FPS: 87.6
FPS: 92.7
FPS: 91.6
FPS: 90.6
FPS: 91.5
FPS: 87.8
FPS: 87.7
FPS: 89.9
FPS: 87.1
FPS: 92.9
FPS: 91.8
FPS: 91.0
FPS: 83.2
FPS: 81.3
FPS: 81.5
FPS: 88.7
FPS: 91.6
FPS: 90.1
FPS

[swscaler @ 0x1208e0000] [swscaler @ 0x141860000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1208e0000] [swscaler @ 0x1208f0000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1208e0000] [swscaler @ 0x120900000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1208e0000] [swscaler @ 0x120910000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1208e0000] [swscaler @ 0x120920000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1208e0000] [swscaler @ 0x120930000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1208e0000] [swscaler @ 0x120940000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1208e0000] [swscaler @ 0x120950000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1208e0000] [swscaler @ 0x120960000] No accelerated colorsp

Detected language in segment 0: en
Detected language in segment 1: en
Detected language in segment 2: en
Detected language in segment 3: en
Detected language in segment 4: en
Detected language in segment 5: en
Detected language in segment 6: en
Detected language in segment 7: en
Detected language in segment 8: en
Detected language in segment 9: en
Detected language in segment 10: en
Detected language in segment 11: en
Detected language in segment 12: en
Detected language in segment 13: en
Detected language in segment 14: en
Detected language in segment 15: en
Detected language in segment 16: en
Detected language in segment 17: en
Detected language in segment 18: en
Detected language in segment 19: en
Detected language in segment 20: en
Detected language in segment 21: en
Detected language in segment 22: en
Detected language in segment 23: en
Detected language in segment 24: en
Detected language in segment 25: en
Detected language in segment 26: en
Detected language in segment 27: en
De

...

Detected language in segment 0: en
Detected language in segment 1: en
Detected language in segment 2: en
Detected language in segment 3: en
Detected language in segment 4: en
Detected language in segment 5: en
Detected language in segment 6: en
Detected language in segment 7: en
Detected language in segment 8: en
Detected language in segment 9: en
Detected language in segment 10: en
Detected language in segment 11: en
Detected language in segment 12: en
Detected language in segment 13: en
Detected language in segment 14: en
Detected language in segment 15: en
Detected language in segment 16: en
Detected language in segment 17: en
Detected language in segment 18: en
Detected language in segment 19: en
Detected language in segment 20: en
Detected language in segment 21: en
Detected language in segment 22: en
Detected language in segment 23: en
Detected language in segment 24: en
Detected language in segment 25: en
Detected language in segment 26: en
Detected language in segment 27: en
De

FPS: 110.7
FPS: 110.5
FPS: 111.4
FPS: 111.6
FPS: 112.9
FPS: 112.4
FPS: 110.7
FPS: 106.0
FPS: 111.8
FPS: 111.9
FPS: 114.1
FPS: 105.9
FPS: 101.7
FPS: 114.4
FPS: 110.5
FPS: 110.1
FPS: 113.2
FPS: 103.7
FPS: 109.3
FPS: 112.4
FPS: 110.5
FPS: 106.2
FPS: 117.1
FPS: 108.2
FPS: 114.1
FPS: 108.5
FPS: 107.2
FPS: 109.3
FPS: 116.7
FPS: 105.7
FPS: 119.7
FPS: 114.4
FPS: 117.4
FPS: 113.1
FPS: 119.5
FPS: 108.8
FPS: 111.2
FPS: 115.9
FPS: 111.8
FPS: 104.1
FPS: 108.5
FPS: 101.0
FPS: 93.4
FPS: 104.1
FPS: 106.2
FPS: 111.9
FPS: 115.5
FPS: 108.6
FPS: 106.4
FPS: 106.8
FPS: 108.1
FPS: 111.8
FPS: 116.5
FPS: 111.1
FPS: 112.4
FPS: 115.4
FPS: 98.9
FPS: 111.9
FPS: 109.4
FPS: 107.6
FPS: 102.3
FPS: 114.6
FPS: 112.4
FPS: 110.9
FPS: 105.5
FPS: 117.6
FPS: 104.0
FPS: 116.8
FPS: 110.0
FPS: 109.9
FPS: 106.7
FPS: 109.6
FPS: 116.2
FPS: 112.1
FPS: 105.9
FPS: 110.2
FPS: 106.5
FPS: 114.6
FPS: 110.0
FPS: 116.6
FPS: 111.9
FPS: 111.7
FPS: 105.5
FPS: 113.6
FPS: 105.9
FPS: 113.8
FPS: 104.0
FPS: 108.3
FPS: 107.6
FPS: 105.2
FPS: 107.4
F

FPS: 106.3
FPS: 113.1
FPS: 110.0
FPS: 117.6
FPS: 114.3
FPS: 115.1
FPS: 108.7
FPS: 99.7
FPS: 115.6
FPS: 111.3
FPS: 104.5
FPS: 111.0
FPS: 106.7
FPS: 110.3
FPS: 110.8
FPS: 98.7
FPS: 102.3
FPS: 111.3
FPS: 104.5
FPS: 109.4
FPS: 111.8
FPS: 109.9
FPS: 120.0
FPS: 115.6
FPS: 108.5
FPS: 116.0
FPS: 107.7
FPS: 114.2
FPS: 108.8
FPS: 114.5
FPS: 106.8
FPS: 107.1
FPS: 114.6
FPS: 109.8
FPS: 100.7
FPS: 112.7
FPS: 107.5
FPS: 111.1
FPS: 110.7
FPS: 106.6
FPS: 110.7
FPS: 109.1
FPS: 111.3
FPS: 99.9
FPS: 115.2
FPS: 114.4
FPS: 119.3
FPS: 116.1
FPS: 105.9
FPS: 111.3
FPS: 108.8
FPS: 101.3
FPS: 102.1
FPS: 103.9
FPS: 110.9
FPS: 106.3
FPS: 110.1
FPS: 103.2
FPS: 115.0
FPS: 109.3
FPS: 109.9
FPS: 106.6
FPS: 111.4
FPS: 107.8
FPS: 106.9
FPS: 96.1
FPS: 97.8
FPS: 107.9
FPS: 111.6
FPS: 114.9
FPS: 117.4
FPS: 118.5
FPS: 104.8
FPS: 106.6
FPS: 105.0
FPS: 107.6
FPS: 114.8
FPS: 117.5
FPS: 114.1
FPS: 113.6
FPS: 109.4
FPS: 111.7
FPS: 114.0
FPS: 112.5
FPS: 104.9
FPS: 110.4
FPS: 113.7
FPS: 112.0
FPS: 113.9
FPS: 108.4
FPS: 108.1
FPS:

FPS: 115.6
FPS: 110.1
FPS: 113.6
FPS: 105.0
FPS: 116.2
FPS: 117.6
FPS: 111.1
FPS: 103.9
FPS: 110.9
FPS: 109.0
FPS: 107.8
FPS: 115.8
FPS: 113.9
FPS: 100.1
FPS: 117.3
FPS: 108.8
FPS: 110.3
FPS: 108.9
FPS: 114.6
FPS: 104.3
FPS: 106.3
FPS: 106.2
FPS: 98.7
FPS: 95.3
FPS: 110.4
FPS: 109.5
FPS: 107.6
FPS: 109.8
FPS: 112.9
FPS: 104.7
FPS: 106.4
FPS: 105.9
FPS: 118.2
FPS: 113.3
FPS: 111.7
FPS: 109.4
FPS: 111.3
FPS: 113.3
FPS: 112.5
FPS: 102.0
FPS: 103.7
FPS: 110.1
FPS: 116.5
FPS: 107.2
FPS: 107.2
FPS: 112.0
FPS: 105.9
FPS: 117.3
FPS: 115.4
FPS: 105.2
FPS: 110.3
FPS: 118.6
FPS: 112.5
FPS: 114.7
FPS: 110.2
FPS: 114.4
FPS: 109.7
FPS: 108.2
FPS: 108.5
FPS: 113.8
FPS: 107.4
FPS: 114.2
FPS: 110.8
FPS: 116.7
FPS: 114.4
FPS: 112.0
FPS: 113.2
FPS: 113.2
FPS: 106.9
FPS: 103.7
FPS: 113.9
FPS: 120.6
FPS: 108.2
FPS: 109.7
FPS: 105.6
FPS: 114.9
FPS: 113.3
FPS: 107.8
FPS: 110.6
FPS: 106.2
FPS: 99.5
FPS: 102.0
FPS: 115.0
FPS: 111.0
FPS: 115.8
FPS: 109.7
FPS: 107.7
FPS: 110.7
FPS: 112.6
FPS: 101.1
FPS: 107.5
FP

FPS: 108.3
FPS: 107.7
FPS: 113.4
FPS: 106.4
FPS: 107.1
FPS: 111.1
FPS: 116.9
FPS: 104.9
FPS: 109.9
FPS: 107.7
FPS: 105.9
FPS: 97.0
FPS: 63.0
FPS: 114.0
FPS: 72.4
FPS: 115.7
FPS: 110.0
FPS: 114.0
FPS: 107.5
FPS: 112.8
FPS: 102.7
FPS: 67.8
FPS: 112.5
FPS: 106.4
FPS: 112.7
FPS: 105.9
FPS: 115.6
FPS: 114.7
FPS: 114.3
FPS: 118.4
FPS: 112.9
FPS: 114.2
FPS: 104.4
FPS: 114.2
FPS: 110.4
FPS: 109.1
FPS: 108.0
FPS: 110.0
FPS: 112.5
FPS: 109.1
FPS: 105.4
FPS: 99.2
FPS: 116.6
FPS: 105.0
FPS: 107.6
FPS: 107.1
FPS: 107.8
FPS: 107.7
FPS: 112.6
FPS: 121.8
FPS: 105.9
FPS: 114.4
FPS: 100.2
FPS: 121.1
FPS: 111.6
FPS: 114.2
FPS: 112.2
FPS: 112.9
FPS: 111.7
FPS: 100.5
FPS: 111.5
FPS: 104.6
FPS: 108.5
FPS: 113.3
FPS: 104.6
FPS: 117.0
FPS: 103.9
FPS: 110.3
FPS: 109.4
FPS: 116.3
FPS: 106.4
FPS: 115.4
FPS: 108.3
FPS: 115.0
FPS: 105.6
FPS: 108.4
FPS: 115.3
FPS: 109.1
FPS: 109.8
FPS: 110.4
FPS: 110.8
FPS: 107.8
FPS: 109.6
FPS: 120.0
FPS: 112.7
FPS: 109.6
FPS: 106.7
FPS: 113.0
FPS: 103.6
FPS: 113.3
FPS: 106.7
FPS:

## Data Manipulation

In [5]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('stopwords')

def read_csv(path: str):
    csv = pd.read_csv(path)
    return csv.head()

titoli = leggi_titoli_video()
os.makedirs('./processed_text', exist_ok=True)    

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/francesconicotra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/francesconicotra/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/francesconicotra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/francesconicotra/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesconicotra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from nltk.tokenize import sent_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

# Scaricare i modelli di punteggiatura per la suddivisione in frasi
nltk.download('punkt')

for titolo in titoli:
    df = read_csv('./transcriptions/' + titolo + '.csv')
    # Crea una lista per memorizzare le frasi suddivise
    frasi = []

    # Itera su ogni riga della colonna 'testo'
    for testo in df['testo']:
        testo.replace('"','')
        # Suddividi il testo in frasi
        frasi.extend(sent_tokenize(testo))

    # Creare un nuovo DataFrame con le frasi suddivise
    df_frasi = pd.DataFrame(frasi, columns=['frase'])

    # Salva le frasi suddivise in un nuovo file CSV
    df_frasi.to_csv('./processed_text/' + titolo + '.csv', index=False)


media parole: 13.76923076923077
media caratteri: 74.3076923076923
media parole: 11.561643835616438
media caratteri: 63.945205479452056
media parole: 15.037593984962406
media caratteri: 85.41353383458646
media parole: 25.435294117647057
media caratteri: 143.12941176470588
media parole: 12.146596858638743
media caratteri: 66.72251308900523
media parole: 16.43558282208589
media caratteri: 91.37423312883436
media parole: 14.794871794871796
media caratteri: 83.52991452991454


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/francesconicotra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string))
    a = [wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)]
    return " ".join(a)

def stopword(string):
    a = [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

def analyze_sentiment(text):
    if isinstance(text,str):
        sia = SentimentIntensityAnalyzer()
        sentiment_score = sia.polarity_scores(text)['compound']
        if sentiment_score >= 0.05:
            return 'positivo'
        elif sentiment_score <= -0.05:
            return 'negativo'
        else:
            return 'neutro'
    else:
        return "neutro"

In [8]:
from transformers import pipeline
import torch
import pandas as pd
torch.set_default_dtype(torch.float32)
from groq import Groq
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv('.env')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
client = Groq(
    api_key=GROQ_API_KEY,
)

# Funzione per leggere il testo dal file CSV
def leggi_testo_da_csv(file_csv):
    df = pd.read_csv(file_csv)
    testo_intervista = " ".join(df['frase'].tolist())
    return testo_intervista

# Funzione per comprendere il contesto e generare domande
def genera_domande(testo, numero_domande=5):
    chat_completion = client.chat.completions.create(
    messages=[
            {
                "role": "user",
                "content": f'Act as if you were a journalist, based on this text:\'{testo}\' create only {numero_domande} questions pertaining to this text and not the answers',
            }
        ],
        model="llama3-8b-8192",
    )
    return chat_completion.choices[0].message.content

In [12]:
wl = WordNetLemmatizer()
os.makedirs('./new_questions', exist_ok=True)    

pt = None
questions = None
questions_list = []

for titolo in titoli:
    path = './processed_text/' + titolo + '.csv'
    pt = read_csv(path)
    pt['clean_text'] = pt['frase'].apply(lambda x: str.lower(x) if pd.isna(x) != True else x)
    pt['clean_text'] = pt['clean_text'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x) if pd.notna(x) else x)
    pt['clean_text'] = pt['clean_text'].apply(lambda x: re.sub(r'[\n\t]', ' ', x) if pd.notna(x) else x)
    pt['clean_text'] = pt['clean_text'].apply(lambda x: lemmatizer(x) if pd.isna(x) != True else x)
    pt['clean_text'] = pt['clean_text'].apply(lambda x: stopword(x) if pd.isna(x) != True else x)
    pt['clean_text'] = pt['clean_text'].apply(lambda x: x.replace('"', ''))
    pt['sentiment'] = pt['clean_text'].apply(analyze_sentiment)
    frasi = pt['frase']
    generated_questions = []
    interview_text = leggi_testo_da_csv(path)
    questions = genera_domande(interview_text)
    # Separazione della stringa in una lista utilizzando una regex
    questions_list.append(re.split(r'\d+\.\s', questions)[1:])  # Usa la regex per separare

In [23]:
import json
# Creazione di un file JSON e scrittura dei dati
with open('comments.json', 'w') as json_file:
    json.dump(comments_json, json_file, indent=4)